#Task 1: Tokenization



In [1]:
# Download gutenberg dataset
# This is the dataset on which all your models will be trained
# https://drive.google.com/file/d/0B2Mzhc7popBga2RkcWZNcjlRTGM/edit

In [2]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)
import glob
import os

In [3]:
import sys, re

path = "/home/nikita/Downloads/Gutenberg/test/"
text = " "
for filename in glob.glob(os.path.join(path, '*.txt')):
    text = text + open(filename,encoding = "ISO-8859-1").read()
    
text = text.replace('\n',' ')

def tokenise(sentence):
    matchStr = re.compile('(?<=[^A-Z][^A-Z][^A-Z][^A-Z].[.?!]) +(?=[A-Z])')
    return matchStr.split(sentence)

def tokenise_corpus(sentences):
    return [re.split("[^A-Za-z0-9']+", sentence.lower()) for sentence in sentences]

sentences = tokenise(text)
corpus = tokenise_corpus(sentences)

i = 0
for sentence in corpus:
    sentence = list(filter(None, sentence))
    corpus[i] = sentence
    i=i+1

for i in range(0, len(corpus)):
    temp = ['@', '@']
    corpus[i] = temp + corpus[i]
    temp = ['#', '#']
    corpus[i] = corpus[i] + temp

    

corpus

[['@',
  '@',
  "lincoln's",
  'first',
  'inaugural',
  'address',
  'march',
  '4',
  '1861',
  'fellow',
  'citizens',
  'of',
  'the',
  'united',
  'states',
  'in',
  'compliance',
  'with',
  'a',
  'custom',
  'as',
  'old',
  'as',
  'the',
  'government',
  'itself',
  'i',
  'appear',
  'before',
  'you',
  'to',
  'address',
  'you',
  'briefly',
  'and',
  'to',
  'take',
  'in',
  'your',
  'presence',
  'the',
  'oath',
  'prescribed',
  'by',
  'the',
  'constitution',
  'of',
  'the',
  'united',
  'states',
  'to',
  'be',
  'taken',
  'by',
  'the',
  'president',
  'before',
  'he',
  'enters',
  'on',
  'the',
  'execution',
  'of',
  'his',
  'office',
  'i',
  'do',
  'not',
  'consider',
  'it',
  'necessary',
  'at',
  'present',
  'for',
  'me',
  'to',
  'discuss',
  'those',
  'matters',
  'of',
  'administration',
  'about',
  'which',
  'there',
  'is',
  'no',
  'special',
  'anxiety',
  'or',
  'excitement',
  '#',
  '#'],
 ['@',
  '@',
  'apprehension',

#Language Model and Smoothing


In [4]:
def language_model(n, corpus):
    my_dict = dict()
    
    for sentence in corpus:
        for i in range(0, len(sentence) - (n - 1)):
            if n == 1:
                key = sentence[i]
            else:
                key = tuple(sentence[i : i + n])

            if key in my_dict:
                my_dict[key] += 1
            else:
                my_dict[key] = 1
    
    my_dict = dict(sorted(my_dict.items(), key = lambda x : x[1], reverse = True))
    return my_dict

trigrams = language_model(3, corpus)
bigrams = language_model(2, corpus)
unigrams = language_model(1, corpus)

del bigrams[('@', '@')] ## start of setence
del bigrams[('#', '#')] ##end of sentence
del unigrams['@'] ##start of sentence
del unigrams['#'] ##end of sentence

In [5]:
##top 10 tri grams
list(trigrams.items())[0:10]

[(('@', '@', 'the'), 47),
 (('@', '@', 'it'), 27),
 (('@', '@', 'if'), 22),
 (('@', '@', 'but'), 20),
 (('@', '@', 'i'), 18),
 (('@', 'it', 'is'), 16),
 (('it', '#', '#'), 16),
 (('@', '@', 'you'), 16),
 (('@', '@', 'he'), 15),
 (('@', '@', 'this'), 14)]

In [6]:
##bottom 10 tri grams
list(trigrams.items())[-10:]

[(('they', 'had', 'heard'), 1),
 (('had', 'heard', 'the'), 1),
 (('heard', 'the', 'discourse'), 1),
 (('the', 'discourse', 'and'), 1),
 (('discourse', 'and', 'enjoyed'), 1),
 (('and', 'enjoyed', 'it'), 1),
 (('enjoyed', 'it', 'so'), 1),
 (('it', 'so', 'much'), 1),
 (('so', 'much', '#'), 1),
 (('much', '#', '#'), 1)]

In [7]:
iplot([{"x" : ['_'.join(x) for x in list(zip(*list(trigrams.items())))[0]], "y": list(zip(*list(trigrams.items())))[1]}])

In [8]:
def complete_ngram(ngram, n):
    ## n is size of n gram
    ## n gram is list of n - 1 grams
    my_dict = {}
    gram = list(trigrams.items())
    for i in gram:
        if((i[0][0] == ngram[n - 3]) and (i[0][1] == ngram[n - 2])):
            key = i[0][2]
            if key in my_dict:
                my_dict[key] += i[1]
            else:
                my_dict[key] = i[1]
    
    my_dict = sorted(my_dict.items(), key = lambda x : x[1], reverse = True)
    return my_dict

ans = complete_ngram(['for', 'a'], 3)
if ans:
    iplot([{"x" : list(zip(*ans))[0], "y": list(zip(*ans))[1]}])

In [9]:
def laplace_smoothing(n_grams): ##for trigrams
    my_dict = dict()
    
    for i in n_grams:
        deno = 0
        for j in n_grams:
            if i[0] == j[0] and i[1] == j[1]:
                deno = deno + n_grams[j] + 1
        my_dict[i] = (n_grams[i] + 1)/deno
        
    return my_dict

laplace_smoothing(trigrams)

{('@', '@', 'the'): 0.08333333333333333,
 ('@', '@', 'it'): 0.04861111111111111,
 ('@', '@', 'if'): 0.03993055555555555,
 ('@', '@', 'but'): 0.036458333333333336,
 ('@', '@', 'i'): 0.03298611111111111,
 ('@', 'it', 'is'): 0.4722222222222222,
 ('it', '#', '#'): 1.0,
 ('@', '@', 'you'): 0.029513888888888888,
 ('@', '@', 'he'): 0.027777777777777776,
 ('@', '@', 'this'): 0.026041666666666668,
 ('@', '@', 'there'): 0.024305555555555556,
 ('@', '@', 'in'): 0.022569444444444444,
 ('@', '@', 'a'): 0.020833333333333332,
 ('them', '#', '#'): 1.0,
 ('@', '@', 'they'): 0.019097222222222224,
 ('failure', '#', '#'): 1.0,
 ('@', '@', 'and'): 0.017361111111111112,
 ('literature', '#', '#'): 1.0,
 ('i', 'do', 'not'): 0.5,
 ('@', 'there', 'is'): 0.4444444444444444,
 ('@', '@', 'to'): 0.013888888888888888,
 ('of', 'the', 'union'): 0.043243243243243246,
 ('it', 'is', 'a'): 0.1038961038961039,
 ('@', 'this', 'is'): 0.36363636363636365,
 ('there', 'is', 'no'): 0.3684210526315789,
 ('is', 'to', 'be'): 0.2258

In [10]:
def good_turing(n_grams):
    max_freq = n_grams.get(max(n_grams, key = n_grams.get))
    lookup_table = dict()
    
    for i in range(1, max_freq + 1):
        count = 0
        for x in n_grams:
            if n_grams[x] == i:
                count += 1
        lookup_table[i] = count 
    
    for x in n_grams:
        if n_grams[x] != max_freq:
            n_grams[x] = (n_grams[x] + 1) * ((lookup_table[n_grams[x] + 1]) / (lookup_table[n_grams[x]]))
    
    return n_grams

def get_probability(word, n_grams):
    num = 10**-7
    deno = 0
    for i in n_grams:
        if word == i:
            num = n_grams[i]
    for i in n_grams:
        if len(word) == 3:
            if i[0] == word[0] and i[1] == word[1]:
                deno = deno + n_grams[i]
        elif len(word) == 2:
            if i[0] == word[0]:
                deno = deno + n_grams[i]
        else:
            deno = len(n_grams)
            break
    
    if deno == 0:
        deno = 1
        
    return num/deno

def backoff(gram):
    prob = 1
    n = 3
    if gram in trigrams:
        prob = get_probability(gram, trigrams)
    elif gram[0:-1] in bigrams:
        prob = get_probability(gram[0:-1], bigrams)
    elif gram[0] in unigrams:
        prob = get_probability(gram[0], unigrams)
    else:
        prob  = 10**-7
    return prob
  

#Task 2: Unigrams and Spelling Correction

In [11]:
def char_gram(gram, n):
    my_dict = dict()
    
    for lst in gram:
        for word in lst:
            temp_word = re.sub("[^a-zA-Z]+", "", word)
            if temp_word != '':
                temp_word = '@@' + temp_word
                temp_word = temp_word + '##'
                for i in range(0, len(temp_word) - (n - 1)):
                    key = tuple(temp_word[i : i + n])
                    if key in my_dict:
                        my_dict[key] += 1
                    else:
                        my_dict[key] = 1
    return my_dict    

char_trigrams = char_gram(corpus, 3)
prob_char_trigrams = laplace_smoothing(char_trigrams)

In [12]:
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

def P(word): 
    prob = 1;
    n = 3;
    for i in range(0, len(word) - (n - 1)):
        key = tuple(word[i : i + n])
        if key in prob_char_trigrams:
            prob = prob * (prob_char_trigrams[key])  
        else:
            prob = 0;
    return prob

def correction(word):
    new_set = {}
    edit = {}
    new_set[word] = P(word)
    edit = edits1(word)
    
    for i in edit:
        i = '@@' + i + '##'
        key = i[2:-2]
        if key not in new_set:
            new_set[key] = P(key)/(10 * max(10 * abs(len(word) - len(key)), 1))
    
    edit = edits2(word)
    for i in edit:
        i = '@@' + i + '##'
        key = i[2:-2]
        if key not in new_set:
            new_set[key] = P(key)/(1000 * max(10 * abs(len(word) - len(key)), 1))
            
    return new_set

def spell_checker(word):
    new_words = correction(word)
    new_words = sorted(new_words.items(), key = lambda x : x[1], reverse = True)
    return new_words[0:5]

In [13]:
spell_checker('direcl')

[('ddrectly', 1.4096435507822839e-05),
 ('eirectly', 7.809370314470273e-06),
 ('drectly', 5.839951853240891e-06),
 ('directly', 5.549219468461482e-06),
 ('didectly', 3.433051852534546e-06)]

#Task 3 : Grammaticality Test

In [14]:
def score_grammaticality(sentence, language_model):
  # Given a sentence, Build a model from the data which can give a score of grammaticality. 
  # More grammatical the sentence, better the score
  # Feel free to add helper functions
    n = 3
    prob = 1
    prob2 = 1
    sentence = re.split("[^A-Za-z0-9\.']+", sentence)
    temp = ['@', '@']
    sentence = temp + sentence
    temp = ['#', '#']
    sentence = sentence + temp
    
    for i in range(0, len(sentence) - (n - 1)):
        temp = tuple(sentence[i : i + n])
        p = get_probability(temp, language_model)
        p2 = backoff(temp)
        prob = prob * p
        prob2 = prob2 * p2
        print(temp, prob, prob2)
    return prob, prob2
    
    

In [15]:
score_grammaticality('introduction will do you excellent service in smoothing', trigrams)

('@', '@', 'introduction') 2.252252252252252e-10 1e-07
('@', 'introduction', 'will') 2.252252252252252e-17 9.999999999999998e-15
('introduction', 'will', 'do') 2.252252252252252e-17 9.999999999999998e-15
('will', 'do', 'you') 2.252252252252252e-17 9.999999999999998e-15
('do', 'you', 'excellent') 5.63063063063063e-18 2.4999999999999996e-15
('you', 'excellent', 'service') 5.63063063063063e-18 2.4999999999999996e-15
('excellent', 'service', 'in') 5.63063063063063e-18 2.4999999999999996e-15
('service', 'in', 'smoothing') 2.815315315315315e-18 1.2499999999999998e-15
('in', 'smoothing', '#') 2.815315315315315e-25 4.7528517110266155e-18
('smoothing', '#', '#') 2.815315315315315e-32 1.601904857103679e-21


(2.815315315315315e-32, 1.601904857103679e-21)